### **Step 1: Load data and check what it looks like**

In [1]:
import pandas as pd
df = pd.read_csv('food_wishes_from2012.csv')
df.head(3)

,Unnamed: 0,video_id,video_title,upload_date,view_count,like_count,comment_count
0,0,CQPLo8hECWg,Twice Baked Potatoes -- How to Make Fancy Stuf...,2012-12-15,4194592,47213,1950
1,1,TsrTU3CJn2c,Irish Shepherd's Pie - Classic Shepherd Pie fo...,2012-03-05,2730866,44230,2843
2,2,wRtGM3f-UBc,How to Flip Food in a Pan Like a Chef!,2012-10-04,2764323,17837,1382


In [2]:
# looks like there's an unwanted column at the very start
# need to remove it
df.drop(columns=['Unnamed: 0'], inplace=True)

In [3]:
# check the data types of columns
df.dtypes

video_id         object
video_title      object
upload_date      object
view_count        int64
like_count        int64
comment_count     int64
dtype: object

In [4]:
# the 'upload date' coulum should be in 'datetime' format but is currently showing as 'object'
# need to parse it to the correct format
df['upload_date'] = pd.to_datetime(df['upload_date'], format='%Y-%m-%d')

In [5]:
# also would be nice to single out the year, month , year-month and day only for finer analyses
df['upload_year'] = df['upload_date'].dt.year
df['upload_month'] = df['upload_date'].dt.month
df['upload_day'] = df['upload_date'].dt.day
df['upload_yearmonth'] = df['upload_date'].dt.to_period('M')

In [6]:
# check if there are any duplicate records in this dataframe
if df.shape[0] - len(df['video_id'].unique()) > 0:
    print('you have duplicates')
else:
    print('you don\'t have duplicates')

you have duplicates


In [7]:
# looks like there are duplicates
# now need to see what the duplicate records look like
df['duplicates'] = df['video_id'].duplicated(keep=False)
duplicates = df[df['duplicates'] == True]
duplicates.sort_values('video_id')

,video_id,video_title,upload_date,view_count,like_count,comment_count,upload_year,upload_month,upload_day,upload_yearmonth,duplicates
41,4i-blr330n4,"How to ""Turn"" a Mushroom - To Honor Earth Day,...",2012-04-20,283726,4006,277,2012,4,20,2012-04,True
43,4i-blr330n4,"How to ""Turn"" a Mushroom - To Honor Earth Day,...",2012-04-20,283726,4006,277,2012,4,20,2012-04,True
40,8SLbT652IiQ,Pickled Jalapeno Rings - Make Your Own Pickled...,2012-02-03,885253,10949,468,2012,2,3,2012-02,True
42,8SLbT652IiQ,Pickled Jalapeno Rings - Make Your Own Pickled...,2012-02-03,885253,10949,468,2012,2,3,2012-02,True
346,DE8EnTymPso,Asparagus Tart Recipe - How to Make a Savory A...,2014-03-25,187064,4437,301,2014,3,25,2014-03,True
365,DE8EnTymPso,Asparagus Tart Recipe - How to Make a Savory A...,2014-03-25,187064,4437,301,2014,3,25,2014-03,True
363,EVuiTbde80k,Tomato & Mozzarella Salad with Burrata Cheese ...,2014-08-01,214980,5218,492,2014,8,1,2014-08,True
353,EVuiTbde80k,Tomato & Mozzarella Salad with Burrata Cheese ...,2014-08-01,214980,5218,492,2014,8,1,2014-08,True
366,KHxlKnIS7pY,Grilled Stuffed Calamari - Grilled Sausage-Stu...,2014-06-13,398552,8069,438,2014,6,13,2014-06,True
356,KHxlKnIS7pY,Grilled Stuffed Calamari - Grilled Sausage-Stu...,2014-06-13,398552,8069,438,2014,6,13,2014-06,True


In [8]:
# well, looks like some how the API queries gathered a few exactly identical records
# then just keep only one of each unique records
df['duplicates'] = df['video_id'].duplicated(keep='last')
df = df[df['duplicates'] == False]
df.drop(columns=['duplicates'], inplace=True)
# confirm if duplicates are removed
df.shape[0]

1118

In [9]:
# finally, see what the cleaned-up dataframe looks like
df.head(3)

,video_id,video_title,upload_date,view_count,like_count,comment_count,upload_year,upload_month,upload_day,upload_yearmonth
0,CQPLo8hECWg,Twice Baked Potatoes -- How to Make Fancy Stuf...,2012-12-15,4194592,47213,1950,2012,12,15,2012-12
1,TsrTU3CJn2c,Irish Shepherd's Pie - Classic Shepherd Pie fo...,2012-03-05,2730866,44230,2843,2012,3,5,2012-03
2,wRtGM3f-UBc,How to Flip Food in a Pan Like a Chef!,2012-10-04,2764323,17837,1382,2012,10,4,2012-10


In [10]:
df.groupby('upload_yearmonth').sum()

,view_count,like_count,comment_count,upload_year,upload_month,upload_day
upload_yearmonth,,,,,,
2012-01,6046941,96394,8131,24144,12,186
2012-02,16977317,210647,11742,24144,24,168
2012-03,9596025,169435,10616,24144,36,190
2012-04,9023302,124481,7167,18108,36,121
2012-05,8362327,125887,8076,24144,60,164
...,...,...,...,...,...,...
2021-11,2576718,82739,4309,22231,121,156
2021-12,3878854,114517,5527,20210,120,154
2022-01,4732117,125758,6507,26286,13,207


In [11]:
import numpy as np
import datetime
from user_func import nlp_workflow as nlp